<h1><center>Laboratorio N° 03: Pivoting y Grouping Sets

<h2>Parte 01. Escribir consultas que utilizan PIVOT.

1.1. Abrir la base de datos TSQL.

In [5]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00

1.2. Crear un vista de categorías de productos, sus cantidaddes por año.

In [14]:
CREATE VIEW CategoriasPorAnio
AS
SELECT categoryid, YEAR(orderdate) AS Anio, SUM(qty) AS Cantidad
FROM Sales.OrderDetails AS od
JOIN Sales.Orders AS o ON od.orderid = o.orderid
JOIN Production.Products AS p ON od.productid = p.productid
GROUP BY categoryid, YEAR(orderdate);
GO


Commands completed successfully.

Total execution time: 00:00:00.004

1.3. Probar la vista creada.

In [15]:
SELECT *
FROM (
    SELECT categoryid, Anio, Cantidad
    FROM CategoriasPorAnio
) AS Fuente
PIVOT (
    SUM(Cantidad)
    FOR Anio IN ([2013], [2014], [2015], [2016])
) AS TablaPivoteada;
GO


(8 rows affected)

Total execution time: 00:00:00.013

categoryid,2013,2014,2015,2016
1,NULL,NULL,NULL,NULL
2,NULL,NULL,NULL,NULL
3,NULL,NULL,NULL,NULL
4,NULL,NULL,NULL,NULL
5,NULL,NULL,NULL,NULL
6,NULL,NULL,NULL,NULL
7,NULL,NULL,NULL,NULL
8,NULL,NULL,NULL,NULL


1.4. Utilizar PIVOT entre categorías y años de ordenes.

In [17]:
CREATE VIEW VentasProductoPorAnio
AS
SELECT p.productname, YEAR(o.orderdate) AS Anio, SUM(od.qty * od.unitprice) AS TotalVentas
FROM Sales.OrderDetails AS od
JOIN Sales.Orders AS o ON od.orderid = o.orderid
JOIN Production.Products AS p ON od.productid = p.productid
GROUP BY p.productname, YEAR(o.orderdate);
GO


Commands completed successfully.

Total execution time: 00:00:00.002

1.5. Para utilizar UNPIVOT, crear una tabla con el resultado de la consulta con PIVOT anterior.

In [18]:
SELECT *
FROM (
    SELECT productname, Anio, TotalVentas
    FROM VentasProductoPorAnio
) AS Fuente
PIVOT (
    SUM(TotalVentas)
    FOR Anio IN ([2013], [2014], [2015], [2016])
) AS TablaPivoteada;
GO


(77 rows affected)

Total execution time: 00:00:00.026

productname,2013,2014,2015,2016
Product ACRVI,NULL,NULL,NULL,NULL
Product AOZBW,NULL,NULL,NULL,NULL
Product APITJ,NULL,NULL,NULL,NULL
Product AQOKR,NULL,NULL,NULL,NULL
Product ASTMN,NULL,NULL,NULL,NULL
Product BIUDV,NULL,NULL,NULL,NULL
Product BKAZJ,NULL,NULL,NULL,NULL
Product BKGEA,NULL,NULL,NULL,NULL
Product BLCAX,NULL,NULL,NULL,NULL
Product BWRLG,NULL,NULL,NULL,NULL


1.6. Probrar la tabla generada

In [19]:
-- Consultar la vista usando PIVOT (ejemplo con años 2013-2016)
SELECT *
FROM (
    SELECT categoryid, YEAR(orderdate) AS Anio, SUM(od.unitprice * od.qty) AS TotalVentas
    FROM Sales.OrderDetails AS od
    JOIN Sales.Orders AS o ON od.orderid = o.orderid
    JOIN Production.Products AS p ON od.productid = p.productid
    GROUP BY categoryid, YEAR(orderdate)
) AS Fuente
PIVOT (
    SUM(TotalVentas)
    FOR Anio IN ([2013], [2014], [2015], [2016])
) AS TablaPivoteada;


(8 rows affected)

Total execution time: 00:00:00.007

categoryid,2013,2014,2015,2016
1,NULL,NULL,NULL,NULL
2,NULL,NULL,NULL,NULL
3,NULL,NULL,NULL,NULL
4,NULL,NULL,NULL,NULL
5,NULL,NULL,NULL,NULL
6,NULL,NULL,NULL,NULL
7,NULL,NULL,NULL,NULL
8,NULL,NULL,NULL,NULL


1.7. Utilizar UNPIVOT.

In [20]:
-- Simulación de tabla pivotada (puedes usar una tabla o una CTE)
WITH PivotTable AS (
    SELECT categoryid, [2013], [2014], [2015], [2016]
    FROM (
        SELECT categoryid, YEAR(orderdate) AS Anio, SUM(od.unitprice * od.qty) AS TotalVentas
        FROM Sales.OrderDetails AS od
        JOIN Sales.Orders AS o ON od.orderid = o.orderid
        JOIN Production.Products AS p ON od.productid = p.productid
        GROUP BY categoryid, YEAR(orderdate)
    ) AS Fuente
    PIVOT (
        SUM(TotalVentas)
        FOR Anio IN ([2013], [2014], [2015], [2016])
    ) AS TablaPivoteada
)

-- Aplicar UNPIVOT
SELECT categoryid, Anio, TotalVentas
FROM PivotTable
UNPIVOT (
    TotalVentas FOR Anio IN ([2013], [2014], [2015], [2016])
) AS Unpivoted;


(0 rows affected)

Total execution time: 00:00:00.006

categoryid,Anio,TotalVentas


1.8. Limpiar los cambios realizados.


In [21]:
DROP VIEW IF EXISTS CategoriasPorAnio;
DROP VIEW IF EXISTS VentasProductoPorAnio;


Commands completed successfully.

Total execution time: 00:00:00.003

<h2>Parte 02. Escribir consultas que utilizan UNPIVOT.

2.1. Ejecutar la siguiente vistas

2.2. Elaborar una consulta sin utilizar GROUPING SETS.

2.3. Consultar con GROUPING SETS.

2.4. Consultar con CUBE.

2.5. Consultar con ROLLUP

2.6. Utilizar Grouping_ID.

2.7. Finalmente limpiar los cambios

<h2>Ejercicio 03: Escribir consultas que usan las clausulas GROUPING SETS, CUBE y ROLLUP

3.1. Escribir una sentencia SELECT contra la tabla Sales.Customers y traer las columnas country, city
y una columna calculada noofcustomers como una cantidad de clientes. Traer multiples grouping
sets basados en las columnas country y city, la columna country, la columna city, y una columna con un grouping set vacio.

3.2. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Día de la columna orderdate como orderday.
<li>Valor de Total de Ventas utilizando la columna val como salesvalue.
<li>Devolver todas los posibles  grouping sets basados en las columnas orderyear, ordermonth, y orderday.

3.3. Copiar la consulta previa y modificarla para utilizar la clausula ROLLUP en vez de CUBE.

3.4. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Columna Calculada con el alias groupid (usar la función GROUPING_ID con los campos order year y order month como parametros).
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Valor Total de Ventas utilizando la columna val como salesvalue.
<li>Desde Año y mes month forman una jerarquia, retornar todos los grouping sets interesantes basados en las columnas orderyear y ordermonth y ordenar el resultado por groupid, orderyear, and ordermonth